In [23]:
!pip install pyarrow

In [24]:
!python -V

Python 3.9.12


In [33]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer

In [26]:

df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [27]:
num_columns = df.shape[1]
print("Number of columns:", num_columns)

Number of columns: 19


In [28]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])



In [29]:

df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60


std_duration = df['duration'].std()
print("Standard deviation of trip durations:", std_duration)

Standard deviation of trip durations: 42.594351241920904


In [30]:

initial_record_count = len(df)




In [31]:
filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
filtered_record_count = len(filtered_df)

In [32]:
fraction_left = filtered_record_count / initial_record_count
print("Fraction of records left after removing outliers:", fraction_left)

Fraction of records left after removing outliers: 0.9812202822125979


In [34]:
df = df[['PULocationID', 'DOLocationID']]

df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)


In [35]:
data_dicts = df.to_dict(orient='records')

In [36]:
dv = DictVectorizer(sparse=True)
feature_matrix = dv.fit_transform(data_dicts)

In [37]:
num_columns = feature_matrix.shape[1]
num_columns

518